# Record of the number of cards on EN web

https://pkmncards.com/

2022, 12, 12

In [4]:
import bs4
import sys
import requests
import pandas as pd

In [5]:
def getContent(cardId):
    # anti-scraping
    user_agent = "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:68.0) Gecko/20100101 Firefox/68.0"
    url = f'https://pkmncards.com/?s={cardId.replace(" ", "+")}'
    response = requests.get(url, headers={'User-Agent': user_agent})
    if response.status_code == 200:
        # print(response.content.decode('utf-8'))
        return response.content.decode('utf-8')
    else:
        print(f"Fail to get the url [{response.status_code}]")
        return [cardId, response.status_code]

## Pokemon cards

In [10]:
def readPokemon(content):
    # start reading content
    soup = bs4.BeautifulSoup(content, 'html.parser')
    card = soup.find('div', class_='tab text') 
    
    name = card.find('span', class_='name').get_text()
    ruleList = ['V', 'VSTAR', 'Radiant', 'ex']
    nameSplit = name.split()
    for ruleCategory in ruleList:
        if ruleCategory in nameSplit:
            rule = ruleCategory
            break
    hp = int(card.find('span', class_='hp').get_text().split()[0])
    color = card.find('span', class_='color').abbr['title']
    stage = card.find('span', class_='stage').get_text()
    evolveText = card.find('span', class_='evolves')
    evolveInto, evolveFrom = "", ""
    if evolveText:
        if 'Evolves into' in evolveText.get_text():
            evolveInto = evolveText.a.get_text()
        if 'Evolves from' in evolveText.get_text():
            evolveFrom = evolveText.a.get_text()
    weakness = card.find('span', class_='weak').abbr['title']
    resistance = card.find('span', class_='resist').abbr['title']
    retreat = int(card.find('span', class_='retreat').abbr.get_text())
    illus = card.find('span', title='Illustrator').a.get_text()
    setName = card.find('span', title='Set').get_text()
    setNumber = int(card.find('span', class_='number').get_text())
    rarity = card.find('span', class_='rarity').get_text()
    regulationMark = card.find('span', class_='Regulation Mark').a.get_text()

    text = card.find('div', class_='text')
    attack1, attack2 = False, False
    
    attack1cost = []
    attack1name = ""
    attack1damage = 0
    attack1effect = ""
    
    attack2cost = []
    attack2name = ""
    attack2damage = 0
    attack2effect = ""
    
    abilityName = ""
    abilityEffect = ""
    vstarPower = ""
    
    for p in text.find_all('p'):
        if p.find('a'):
            # has ability
            abilityName = p.find('a').next_sibling.split("⇢")[-1].strip()
            abilityEffect = p.get_text().split('\n')[-1].strip()
            if p.get_text().split('\n')[0] == 'VSTAR Power':
                vstarPower = abilityName
        elif not attack1:
            for energy in p.find_all('abbr'):
                attack1cost.append(energy['title'])
            attack1name = p.find_all('span')[-1].get_text()
            if p.find('br'):
                potentialDamage = p.find('br').previous_sibling
                if '0' in potentialDamage:
                    attack1damage = int(potentialDamage.split()[-1])
                    attack1effect = p.find('br').next_sibling.strip()
            else:
                attack1damage = int(p.get_text().split()[-1])
            attack1 = True
        elif not attack2:
            for energy in p.find_all('abbr'):
                attack2cost.append(energy['title'])
            attack2name = p.find_all('span')[-1].get_text()
            if p.find('br'):
                potentialDamage = p.find('br').previous_sibling
                if '0' in potentialDamage:
                    attack2damage = int(potentialDamage.split()[-1])
                    attack2effect = p.find('br').next_sibling.strip()
            else:
                attack2damage = int(p.get_text().split()[-1])
            attack2 = True
        else:
            raise Exception("No more slots for attacks!")

    

In [17]:
cardId = 'Dunsparce FST 207'

content = getContent(cardId)
readPokemon(content)

## Trainer cards

In [13]:
cardId = 'Ultra Ball BRS 150'

content = getContent(cardId)

In [23]:
def readTrainer(content):
    # start reading content
    soup = bs4.BeautifulSoup(content, 'html.parser')
    card = soup.find('div', class_='tab text') 
    name = card.find('span', class_='name').get_text()
    category = card.find('span', class_='sub-type').get_text()
    effect = card.find('div', class_='text').get_text().strip()
    illus = card.find('span', title='Illustrator').a.get_text()
    setName = card.find('span', title='Set').get_text()
    setNumber = int(card.find('span', class_='number').get_text())
    rarity = card.find('span', class_='rarity').get_text()
    regulationMark = card.find('span', class_='Regulation Mark').a.get_text()

In [24]:
cardId = 'Ultra Ball BRS 150'

content = getContent(cardId)
readTrainer(content)

## Energy cards

In [18]:
cardId = 'Hiding Darkness Energy DAA 175'
content = getContent(cardId)

In [19]:
soup = bs4.BeautifulSoup(content, 'html.parser')
card = soup.find('div', class_='tab text') 

In [20]:
name = card.find('span', class_='name').get_text()
category = card.find('span', class_='sub-type').get_text()
effect = card.find('div', class_='text').get_text().strip()
name, category, effect

('Hiding {D} Energy',
 'Special Energy',
 'As long as this card is attached to a Pokémon, it provides {D} Energy.\nThe {D} Pokémon this card is attached to has no Retreat Cost.')

In [14]:
illus = card.find('span', title='Illustrator').a.get_text()
setName = card.find('span', title='Set').get_text()
setNumber = int(card.find('span', class_='number').get_text())
rarity = card.find('span', class_='rarity').get_text()
regulationMark = card.find('span', class_='Regulation Mark').a.get_text()
illus, setName, setNumber, rarity, regulationMark

('n/a', 'Darkness Ablaze', 176, 'Uncommon', 'D')

## Experiment

- Board (1 Pokemon): Lugia V
- Hand (3 cards): Lugia V, Ultra ball, Evolution Incense
- Deck (4 cards): Archeops, Lugia VSTAR, Double Turbo Energy, Double Turbo Energ

In [2]:
deckList = """Pokémon (4)
2 Lugia V SIT 138
1 Lugia VSTAR SIT 139
1 Archeops SIT 147

Trainer (2)
1 Ultra Ball BRS 150
1 Evolution Incense SSH 163

Energy (2)
2 Double Turbo Energy BRS 151
"""

In [17]:
deckList = deckList.replace("Powerful Colorless Energy", "Powerful Energy")

In [1]:
import EnumerateBoard as eb

In [2]:
b = eb.Board('sample-deck-8.txt')

Pokemon cards loaded. 4 cards in deck.
Trainer cards loaded. 6 cards in deck.
Energy cards loaded. 8 cards in deck.


In [3]:
b.snapshot()

Active:

Bench:

Hand:

Prize cards:

Deck:
Lugia V 1
Lugia V 2
Lugia VSTAR 3
Archeops 4
Ultra Ball 5
Evolution Incense 6
Double Turbo Energy 7
Double Turbo Energy 8

Discard pile:

Lost zone:


In [4]:
b.shuffleDeck()

In [5]:
b.snapshot()

Active:

Bench:

Hand:

Prize cards:

Deck:
Archeops 4
Evolution Incense 6
Lugia V 2
Double Turbo Energy 8
Double Turbo Energy 7
Ultra Ball 5
Lugia VSTAR 3
Lugia V 1

Discard pile:

Lost zone:
